In [3]:
import pandas as pd
import hplib as hpl
import hplib_database as db
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Add 8 rows for own heatpump in output/database_heating.csv

In [ ]:
# Afterwards process
# Reduce to climate measurement series with average climate, delete redundant entries and save to csv sub-database
db.reduce_heating_data('database_heating.csv','average')
# -> this creates /output/database_heating_average.csv
db.normalize_heating_data('database_heating_average.csv')
# -> this creates /output/database_heating_average_normalized.csv
db.identify_subtypes('database_heating_average_normalized.csv')
# -> this creates /output/database_heating_average_normalized_subtypes.csv
db.calculate_heating_parameters('database_heating_average_normalized_subtypes.csv')
# -> this creates /output/hplib_database_heating.csv
# -> this creates /hplib/hplib_database.csv

In [6]:
parameters = hpl.get_parameters('Example-Model')
parameters

,Manufacturer,Model,P_th_h_ref [W],P_el_h_ref [W],COP_ref,Group,p1_P_th [1/°C],p2_P_th [1/°C],p3_P_th [-],p4_P_th [1/°C],p1_P_el_h [1/°C],p2_P_el_h [1/°C],p3_P_el_h [-],p4_P_el_h [1/°C],p1_COP [-],p2_COP [-],p3_COP [-],p4_COP [-]
0,Example-Manufacturer,Example-Model,4500,1810,2.486188,1.0,-213.676356,0.01171,0.377844,213.651329,70.622273,0.02074,-0.155403,-70.64454,212.835788,-0.07804,7.072106,-212.702424


In [11]:
# Read input file with temperatures
df = pd.read_csv('../input/TestYear.csv')
df['T_amb'] = df['T_in_primary'] # air/water heat pump -> T_amb = T_in_primary
# Simulate with values
# Load parameters
parameters = hpl.get_parameters(model='Example-Model')
# Create heat pump object with parameters
heatpump = hpl.HeatPump(parameters)
# whereas mode = 1 is for heating and mode = 2 is for cooling
results = heatpump.simulate(t_in_primary=df['T_in_primary'].values, t_in_secondary=df['T_in_secondary'].values, t_amb=df['T_amb'].values, mode=1)
# Plot / Print some results
# example for distribution of COPs
results=pd.DataFrame.from_dict(results)
# Calclulate seasonal performance factor (SPF)
SPF = results['P_th'].mean() / results['P_el'].mean() 
print('The seasonal performance factor (SPF) for one year is = '+str(round(SPF,1)))

The seasonal performance factor (SPF) for one year is = 4.4
